In [1]:
import sys

sys.path.append("/media/cyprien/Data/Documents/Github/pytorch-forecasting")

import pytorch_lightning as pl
import seaborn as sns
import torch
from data_factory.dataLoader import StockPricesLoader
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import notebook

from utilities import *

from tqdm.notebook import tqdm

%matplotlib inline
sns.set_style("whitegrid")
notebook.tqdm().pandas()

pl.seed_everything(42)

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


0it [00:00, ?it/s]

Global seed set to 42


42

In [ ]:
config = load_config("../config/config.yml")

assert config['model'] == 'temporal_fusion_transformer', 'Invalid model in file configuration for this script'

dl = StockPricesLoader(use_previous_files=True)
if not dl.initialized:
    dl.init()

INFO:data_factory.dataLoader:Use init() method to load the data
INFO:data_factory.dataLoader:../data/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv loaded. shape (2332531, 12)
INFO:data_factory.dataLoader:../data/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv loaded. shape (112000, 12)
INFO:data_factory.dataLoader:Missing targets dropped
INFO:data_factory.dataLoader:ExpectedDividend filled with 0. other values filled with ffill
INFO:data_factory.dataLoader:adjust prices
 90%|████████▉ | 1793/2000 [05:32<00:49,  4.17it/s]

## TemporalFusionTransformer

In [ ]:
from pytorch_forecasting.models import TemporalFusionTransformer
import logging
from logging import WARNING
logging.basicConfig(level=WARNING)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard


args = dict(
    hidden_size=16, #config['temporal_fusion_transformer']['hidden_size'],
    lstm_layers=1, #config['temporal_fusion_transformer']['lstm_layers'],
    dropout=config['temporal_fusion_transformer']['dropout'],
    attention_head_size=4,#config['temporal_fusion_transformer']['attention_head_size']
    # output_size=dl.max_prediction_length,
    # max_encoder_length=dl.max_encoder_length
)

# configure network and trainer
trainer = pl.Trainer(
    accelerator='gpu',
    gradient_clip_val=0.1,
    # clipping gradients is a hyperparameter and important to prevent divergence
    # of the gradient for recurrent neural networks
    auto_lr_find=True,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    weights_summary="top",
)

model = TemporalFusionTransformer.from_dataset(
    dl.df_train_timeseries,
    **args
)


print(f"Number of parameters in network: {model.size() / 1e3:.1f}k")

#### Train the model

In [ ]:
## fit network

fit = True

if fit:
    trainer.fit(model, train_dataloaders=dl.train_dl, val_dataloaders=dl.val_dl)
else:
    model = TemporalFusionTransformer.load_from_checkpoint('././lightning_logs/lightning_logs/version_5/checkpoints/epoch=0-step=49915.ckpt')


### Results

In [ ]:
import numpy as np

actuals = torch.cat([y for _, (y, _) in tqdm(iter(dl.test_dl))])
predictions = trainer.predict(model, (X for X, (y, _) in dl.test_dl))

predictions_np = np.array([i.prediction.numpy() for i in predictions]).squeeze(axis=3).reshape(-1, 10)
actuals_np = actuals.cpu().detach().numpy()

In [ ]:
actuals_unscaled = actuals_np.reshape(dl.df_train_ppc.SecuritiesCode.unique().size, -1, 10).copy()
predictions_unscaled = predictions_np.reshape(dl.df_train_ppc.SecuritiesCode.unique().size, -1, 10).copy()

for i, scaler in enumerate(dl.scalers):
    actuals_unscaled[i] = scaler.inverse_transform(actuals_unscaled[i])
for i, scaler in enumerate(dl.scalers):
    predictions_unscaled[i] = scaler.inverse_transform(predictions_unscaled[i])

actuals_unscaled = actuals_unscaled.reshape(-1, 10)
predictions_unscaled = predictions_unscaled.reshape(-1, 10)

In [ ]:

df_test_res = dl.df_test_ppc.copy()

df_test_res['close_true'] = actuals_np[:, 0].flatten()
df_test_res['close_pred'] = predictions_np[:, 0].flatten()

df_test_res['close_true_unscaled'] = actuals_unscaled[:, 0].flatten()
df_test_res['close_pred_unscaled'] = predictions_unscaled[:, 0].flatten()

df_test_res['target_true'] = (actuals_np[:, 1] - actuals_np[:, 0]) / actuals_np[:, 0]
df_test_res['target_pred'] = (predictions_np[:, 1] - predictions_np[:, 0]) / predictions_np[:, 0]

df_test_res['target_true_unscaled'] = (actuals_unscaled[:, 1] - actuals_unscaled[:, 0]) / actuals_unscaled[:, 0]
df_test_res['target_pred_unscaled'] = (predictions_unscaled[:, 1] - predictions_unscaled[:, 0]) / predictions_unscaled[:, 0]

df_test_res.loc[:, ['Close', 'close_true_unscaled', 'close_pred_unscaled', 'Target', 'target_true_unscaled', 'target_pred_unscaled']]


In [ ]:
import matplotlib.pyplot as plt

for sc in dl.df_train_ppc.SecuritiesCode.unique()[:3]:
    df = df_test_res[df_test_res.SecuritiesCode == sc]
    figure = plt.figure(figsize=(20, 5))
    # plt.plot(df.Date, df.Close, label='close true', figure=figure)
    plt.plot(df.Date, df.close_true_unscaled, label='close true', figure=figure)
    plt.plot(df.Date, df.close_pred_unscaled, label='close pred', figure=figure)

    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

recreated_target_is_valid = df_test_res.groupby('SecuritiesCode').apply(lambda x: (x.authentic == True).shift(-2).fillna(value=False) & (x.authentic == True).shift(-1).fillna(value=False) & (x.authentic == True)).reset_index(drop=True)
evaluated_target = (df_test_res.authentic == True)

for sc in df_test_res.SecuritiesCode.unique()[:8]:
    df = df_test_res[(df_test_res.SecuritiesCode == sc) & evaluated_target]
    figure = plt.figure(figsize=(20, 5))
    plt.scatter(df.Date, df.Target, label='true', figure=figure, alpha=.5)
    # plt.scatter(df.Date, df.target_true_unscaled, label='true2', figure=figure, alpha=.5)
    plt.scatter(df.Date, df.target_pred_unscaled, label='pred', figure=figure, alpha=.5)
    plt.legend()
    plt.show()

In [ ]:
from utilities.evaluation import calc_spread_return_sharpe

In [ ]:
df_test_res['Rank'] = (df_test_res.groupby("Date")["target_true_unscaled"].rank(ascending=False, method="first") - 1).astype(int)
calc_spread_return_sharpe(df_test_res)

In [ ]:
df_test_res['Rank'] = (df_test_res.groupby("Date")["target_pred_unscaled"].rank(ascending=False, method="first") - 1).astype(int)
calc_spread_return_sharpe(df_test_res)